In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
#Imports
from google import genai
from tavily import TavilyClient
import os
import re
import json
import sqlite3
from datetime import datetime
from typing import List, Dict, Any, Optional
import numpy as np
from pathlib import Path
from typing import Callable, Any, Dict, Optional
from memory_system import (
    MemoryStore, MemoryRetriever, FeedbackHandler, 
    ContextBuilder, AgentMemoryIntegration
)
from reflection_system import ReflectionWorkflow

In [4]:
# Create client
client = genai.Client(api_key=os.environ.get('GOOGLE_API_KEY'))
model="gemini-2.5-pro"
tavily_client = TavilyClient(api_key=os.environ.get('TAVILY_API_KEY')) # Initialize Tavily client

# Memory

# Initialize memory system
memory_store = MemoryStore("pm_agent_memory.db")
retriever = MemoryRetriever(memory_store)
feedback_handler = FeedbackHandler(memory_store)
context_builder = ContextBuilder(retriever)
memory_integration = AgentMemoryIntegration(
    memory_store, retriever, feedback_handler, context_builder
)

In [15]:
#Agents

class StatisticianAgent:
    """Statistician Agent that provides statistical validation, methodology suggestions, and risk analysis."""
    
    def __init__(self, model: str, client, memory_integration: AgentMemoryIntegration, num_reflections: int = 0):
        self.model = model
        self.client = client
        self.memory_integration = memory_integration
        self.conversation_history = []
        self.num_reflections = num_reflections
        self.reflection_workflow = ReflectionWorkflow(client, model, num_reflections)
        self.system_instruction = """You are an expert Statistician and Data Scientist with deep expertise in:
        - Experimental design and A/B testing methodologies
        - Causal inference techniques (propensity score matching, instrumental variables, difference-in-differences, RCTs)
        - Statistical hypothesis testing and power analysis
        - Observational study design and confounding control
        - Sample size calculations and statistical power
        - Bayesian and frequentist inference
        - Time series analysis and longitudinal data methods
        - Machine learning validation and model evaluation
        - Data quality assessment and bias detection
        - Privacy-preserving statistical methods
        - Communicating statistical concepts to non-technical stakeholders

        You analyze product plans, research designs, and data strategies to:
        1. Validate statistical assumptions and methodological soundness
        2. Suggest appropriate statistical techniques and experimental designs
        3. Flag potential risks, biases, and confounding factors
        4. Provide specific, actionable recommendations with clear rationale
        5. Assess feasibility of causal claims and required data infrastructure
        
        You communicate complex statistical concepts clearly, always considering practical constraints
        like sample size, data collection feasibility, and computational resources."""
    
    def __call__(self, analysis_request: str, num_reflections: Optional[int] = None, verbose: bool = False):
        """
        Analyze plans/designs from a statistical perspective.
        
        Args:
            analysis_request: The plan, design, or question to analyze (from user, PM, or UX researcher)
            num_reflections: Number of reflections for this specific query
            verbose: If True, returns reflection data
            
        Returns:
            (response, memory_id) tuple or (response, memory_id, reflection_data) if verbose
        """
        reflections_to_use = num_reflections if num_reflections is not None else self.num_reflections
        
        # Get memory context
        memory_context = self.memory_integration.pre_run_hook(analysis_request, "statistical_analysis")

        # Check if web search needed for recent statistical methods or research
        decision_prompt = f"""Analyze this statistical analysis request and determine if you need to search for 
recent statistical methodologies, causal inference techniques, or research papers.

Request: {analysis_request}

Respond with ONLY "YES" or "NO":"""

        decision_response = self.client.models.generate_content(
            model=self.model,
            contents=decision_prompt
        )
        
        needs_search = "YES" in decision_response.text.strip().upper()
        
        search_context = ""
        if needs_search:
            print("🔍 Searching for statistical methods and research...")
            search_results = search_web(f"statistical methodology causal inference {analysis_request[:100]}")
            if isinstance(search_results, list):
                search_context = "\n\nRecent Statistical Methods & Research:\n"
                for i, result in enumerate(search_results, 1):
                    search_context += f"\n{i}. {result.get('title', '')}\n{result.get('content', '')}\n"
        
        # Build conversation context
        conversation_context = ""
        if self.conversation_history:
            conversation_context = "\n\nPrevious statistical analyses:\n"
            for entry in self.conversation_history[-3:]:
                conversation_context += f"Request: {entry['request']}\nAnalysis: {entry['response'][:300]}...\n\n"

        def generate_response(query: str) -> str:
            full_query = (
                memory_context +
                conversation_context + 
                f"\nProvide statistical analysis for:\n{query}\n\n"
                "Structure your response with:\n"
                "1. **Validation of Assumptions**: What statistical assumptions are being made?\n"
                "2. **Recommended Techniques**: Which specific statistical methods should be used?\n"
                "3. **Risk Warnings**: What are the key statistical risks and limitations?\n"
                "4. **Implementation Details**: Sample sizes, power calculations, data requirements\n"
                "5. **Causal Inference Considerations**: If applicable, how to establish causality" +
                search_context
            )
            
            response = self.client.models.generate_content(
                model=self.model,
                contents=full_query,
                config={'system_instruction': self.system_instruction}
            )
            return response.text
        
        if reflections_to_use > 0:
            print(f"🤔 Reflecting {reflections_to_use} time(s)...")
            self.reflection_workflow.num_reflections = reflections_to_use
            result = self.reflection_workflow.execute(
                initial_query=analysis_request,
                generate_response_fn=generate_response,
                system_instruction=self.system_instruction
            )
            final_response = result['final_response']
            reflection_data = result
        else:
            final_response = generate_response(analysis_request)
            reflection_data = None

        self.conversation_history.append({
            'request': analysis_request,
            'response': final_response
        })
        
        memory_id = self.memory_integration.post_run_hook(
            analysis_request,
            final_response,
            memory_type="statistical_analysis",
            metadata={
                'search_used': needs_search,
                'num_reflections': reflections_to_use
            }
        )

        if verbose and reflection_data:
            return final_response, memory_id, reflection_data
        
        return final_response, memory_id
    
    def set_reflections(self, num_reflections: int):
        self.num_reflections = num_reflections
        self.reflection_workflow.num_reflections = num_reflections
        print(f"Default reflection count updated to: {num_reflections}")
    
    def apply_feedback(self, memory_id: int, feedback_type: str, feedback_content: str = ""):
        self.memory_integration.apply_feedback(memory_id, feedback_type, feedback_content)
        print(f"Feedback '{feedback_type}' applied to memory {memory_id}")
        
    def clear_history(self):
        self.conversation_history = []
        self.reflection_workflow.clear_history()
        print("Statistical analysis history cleared.")

class UXResearcherAgent:
    """UX Researcher Agent that conducts user research based on product briefs."""
    
    def __init__(self, model: str, client, memory_integration: AgentMemoryIntegration, num_reflections: int = 0):
        self.model = model
        self.client = client
        self.memory_integration = memory_integration
        self.conversation_history = []
        self.num_reflections = num_reflections
        self.reflection_workflow = ReflectionWorkflow(client, model, num_reflections)
        self.system_instruction = """You are an experienced UX Researcher with expertise in:
        - User interviews and surveys
        - Usability testing and evaluation
        - User personas and journey mapping
        - Information architecture
        - Behavioral analysis and user psychology
        - A/B testing and experimentation
        - Accessibility and inclusive design
        - Analytics interpretation
        - Presenting research findings and recommendations

        You analyze product briefs and provide detailed UX research plans, methodologies, 
        and insights. You ask clarifying questions about target users, use cases, and 
        constraints. You base your recommendations on established UX principles and research methods."""
    
    def __call__(self, brief: str, num_reflections: Optional[int] = None, verbose: bool = False):
        """
        Process a product brief and provide UX research recommendations.
        
        Args:
            brief: The product brief (from user or PM agent)
            num_reflections: Number of reflections for this specific query
            verbose: If True, returns reflection data
            
        Returns:
            (response, memory_id) tuple or (response, memory_id, reflection_data) if verbose
        """
        reflections_to_use = num_reflections if num_reflections is not None else self.num_reflections
        
        # Get memory context
        memory_context = self.memory_integration.pre_run_hook(brief, "ux_research")

        # Check if web search needed for UX best practices
        decision_prompt = f"""Analyze this UX research brief and determine if you need to search for current UX best practices, research methodologies, or industry standards.

Brief: {brief}

Respond with ONLY "YES" or "NO":"""

        decision_response = self.client.models.generate_content(
            model=self.model,
            contents=decision_prompt
        )
        
        needs_search = "YES" in decision_response.text.strip().upper()
        
        search_context = ""
        if needs_search:
            print("🔍 Searching for UX research best practices...")
            search_results = search_web(f"UX research methodology {brief[:100]}")
            if isinstance(search_results, list):
                search_context = "\n\nUX Research Best Practices:\n"
                for i, result in enumerate(search_results, 1):
                    search_context += f"\n{i}. {result.get('title', '')}\n{result.get('content', '')}\n"
        
        # Build conversation context
        conversation_context = ""
        if self.conversation_history:
            conversation_context = "\n\nPrevious research context:\n"
            for entry in self.conversation_history[-3:]:
                conversation_context += f"Brief: {entry['brief']}\nResearch Plan: {entry['response']}\n\n"

        def generate_response(query: str) -> str:
            full_query = (
                memory_context +
                conversation_context + 
                f"\n{query}" +
                search_context
            )
            
            response = self.client.models.generate_content(
                model=self.model,
                contents=full_query,
                config={'system_instruction': self.system_instruction}
            )
            return response.text
        
        if reflections_to_use > 0:
            print(f"🤔 Reflecting {reflections_to_use} time(s)...")
            self.reflection_workflow.num_reflections = reflections_to_use
            result = self.reflection_workflow.execute(
                initial_query=brief,
                generate_response_fn=generate_response,
                system_instruction=self.system_instruction
            )
            final_response = result['final_response']
            reflection_data = result
        else:
            final_response = generate_response(brief)
            reflection_data = None

        self.conversation_history.append({
            'brief': brief,
            'response': final_response
        })
        
        memory_id = self.memory_integration.post_run_hook(
            brief,
            final_response,
            memory_type="ux_research",
            metadata={
                'search_used': needs_search,
                'num_reflections': reflections_to_use
            }
        )

        if verbose and reflection_data:
            return final_response, memory_id, reflection_data
        
        return final_response, memory_id
    
    def set_reflections(self, num_reflections: int):
        self.num_reflections = num_reflections
        self.reflection_workflow.num_reflections = num_reflections
        print(f"Default reflection count updated to: {num_reflections}")
    
    def apply_feedback(self, memory_id: int, feedback_type: str, feedback_content: str = ""):
        self.memory_integration.apply_feedback(memory_id, feedback_type, feedback_content)
        print(f"Feedback '{feedback_type}' applied to memory {memory_id}")
        
    def clear_history(self):
        self.conversation_history = []
        self.reflection_workflow.clear_history()
        print("Research history cleared.")

class SampleDataGeneratorAgent:
    """Agent that generates sample/mock data for testing and prototyping."""
    
    def __init__(self, model: str, client, memory_integration: AgentMemoryIntegration, num_reflections: int = 0):
        self.model = model
        self.client = client
        self.memory_integration = memory_integration
        self.conversation_history = []
        self.num_reflections = num_reflections
        self.reflection_workflow = ReflectionWorkflow(client, model, num_reflections)
        self.system_instruction = """You are a Sample Data Generator specialist with expertise in:
        - Creating realistic test data and mock datasets
        - Understanding data schemas and structures
        - Generating diverse, representative data samples
        - Ensuring data privacy and anonymization
        - Creating edge cases and boundary conditions
        - Generating data in various formats (JSON, CSV, SQL, etc.)
        - Synthetic data generation techniques
        - Data volume scaling considerations

        You generate realistic, contextually appropriate sample data based on specifications.
        You ensure data quality, diversity, and adherence to constraints. You can generate
        data for user profiles, transactions, interactions, or any other entities needed
        for testing and prototyping."""
    
    def __call__(self, specification: str, num_reflections: Optional[int] = None, verbose: bool = False):
        """
        Generate sample data based on specification.
        
        Args:
            specification: Description of data to generate (schema, format, volume, etc.)
            num_reflections: Number of reflections for this specific query
            verbose: If True, returns reflection data
            
        Returns:
            (response, memory_id) tuple or (response, memory_id, reflection_data) if verbose
        """
        reflections_to_use = num_reflections if num_reflections is not None else self.num_reflections
        
        memory_context = self.memory_integration.pre_run_hook(specification, "data_generation")

        # Build conversation context
        conversation_context = ""
        if self.conversation_history:
            conversation_context = "\n\nPrevious data generation context:\n"
            for entry in self.conversation_history[-3:]:
                conversation_context += f"Spec: {entry['spec']}\n\n"

        def generate_response(query: str) -> str:
            full_query = (
                memory_context +
                conversation_context + 
                f"\nGenerate sample data based on: {query}\n\n"
                "Provide the data in a structured format with clear labels and explanations."
            )
            
            response = self.client.models.generate_content(
                model=self.model,
                contents=full_query,
                config={'system_instruction': self.system_instruction}
            )
            return response.text
        
        if reflections_to_use > 0:
            print(f"🤔 Reflecting {reflections_to_use} time(s)...")
            self.reflection_workflow.num_reflections = reflections_to_use
            result = self.reflection_workflow.execute(
                initial_query=specification,
                generate_response_fn=generate_response,
                system_instruction=self.system_instruction
            )
            final_response = result['final_response']
            reflection_data = result
        else:
            final_response = generate_response(specification)
            reflection_data = None

        self.conversation_history.append({
            'spec': specification,
            'response': final_response
        })
        
        memory_id = self.memory_integration.post_run_hook(
            specification,
            final_response,
            memory_type="data_generation",
            metadata={'num_reflections': reflections_to_use}
        )

        if verbose and reflection_data:
            return final_response, memory_id, reflection_data
        
        return final_response, memory_id
    
    def set_reflections(self, num_reflections: int):
        self.num_reflections = num_reflections
        self.reflection_workflow.num_reflections = num_reflections
        print(f"Default reflection count updated to: {num_reflections}")
    
    def apply_feedback(self, memory_id: int, feedback_type: str, feedback_content: str = ""):
        self.memory_integration.apply_feedback(memory_id, feedback_type, feedback_content)
        print(f"Feedback '{feedback_type}' applied to memory {memory_id}")
        
    def clear_history(self):
        self.conversation_history = []
        self.reflection_workflow.clear_history()
        print("Data generation history cleared.")

class ProductManagerAgent:
    """Product Manager Agent with conversation memory and long-term learning."""
    
    def __init__(self, model: str, client, memory_integration: AgentMemoryIntegration, num_reflections: int = 0):
        self.model = model
        self.client = client
        self.memory_integration = memory_integration
        self.conversation_history = []
        self.num_reflections = num_reflections
        self.reflection_workflow = ReflectionWorkflow(client, model, num_reflections)
        self.system_instruction = """You are an experienced Product Manager with expertise in:
        - Product strategy and roadmap planning
        - Market research and competitive analysis
        - User research and requirements gathering
        - Feature prioritization and backlog management
        - Stakeholder communication
        - Metrics and KPI definition
        - Go-to-market strategies
        - Agile methodologies
        - Providing feedback to your product specialist, engineering, design, data science, testing, and marketing teams

        You provide practical, actionable advice and ask clarifying questions when needed.
        When provided with search results or past memories, incorporate relevant information into your response and cite sources."""
    
    def __call__(self, user_query: str, num_reflections: Optional[int] = None, verbose: bool = False):
        """
        Make the agent callable like a function.
        
        Args:
            user_query: The user's query
            num_reflections: Number of reflections for this specific query (overrides default)
            verbose: If True, returns reflection data
            
        Returns:
            (response, memory_id) tuple or (response, memory_id, reflection_data) if verbose
        """
        # Use query-specific reflections or fall back to instance default
        reflections_to_use = num_reflections if num_reflections is not None else self.num_reflections
        
        # Get memory context
        memory_context = self.memory_integration.pre_run_hook(user_query, "pm_interaction")

        # Decide if search is needed
        decision_prompt = f"""Analyze this query and determine if you need web search.

Query: {user_query}

Respond with ONLY "YES" or "NO":"""

        decision_response = self.client.models.generate_content(
            model=self.model,
            contents=decision_prompt
        )
        
        needs_search = "YES" in decision_response.text.strip().upper()
        
        # Perform search if needed
        search_context = ""
        if needs_search:
            print("🔍 Searching the web...")
            search_results = search_web(user_query)
            if isinstance(search_results, list):
                search_context = "\n\nWeb Search Results:\n"
                for i, result in enumerate(search_results, 1):
                    search_context += f"\n{i}. {result.get('title', '')}\n{result.get('content', '')}\n"
        
        # Build conversation context
        conversation_context = ""
        if self.conversation_history:
            conversation_context = "\n\nPrevious conversation:\n"
            for entry in self.conversation_history[-5:]:  # Keep last 5 exchanges
                conversation_context += f"User: {entry['user']}\nAssistant: {entry['assistant']}\n\n"

        # Define response generation function for reflection workflow
        def generate_response(query: str) -> str:
            full_query = (
                memory_context +
                conversation_context + 
                f"\n{query}" +
                search_context
            )
            
            response = self.client.models.generate_content(
                model=self.model,
                contents=full_query,
                config={'system_instruction': self.system_instruction}
            )
            return response.text
        
        # Execute with or without reflection
        if reflections_to_use > 0:
            print(f"🤔 Reflecting {reflections_to_use} time(s)...")
            # Temporarily update reflection workflow
            self.reflection_workflow.num_reflections = reflections_to_use
            result = self.reflection_workflow.execute(
                initial_query=user_query,
                generate_response_fn=generate_response,
                system_instruction=self.system_instruction
            )
            final_response = result['final_response']
            reflection_data = result
        else:
            final_response = generate_response(user_query)
            reflection_data = None

        # Store in conversation history
        self.conversation_history.append({
            'user': user_query,
            'assistant': final_response
        })
        
        # Log to long-term memory
        memory_id = self.memory_integration.post_run_hook(
            user_query,
            final_response,
            memory_type="pm_interaction",
            metadata={
                'search_used': needs_search, 
                'num_reflections': reflections_to_use, 
                'reflection_used': reflections_to_use > 0
            }
        )

        if verbose and reflection_data:
            return final_response, memory_id, reflection_data
        
        return final_response, memory_id
    
    def set_reflections(self, num_reflections: int):
        """Update the default number of reflections."""
        self.num_reflections = num_reflections
        self.reflection_workflow.num_reflections = num_reflections
        print(f"Default reflection count updated to: {num_reflections}")
    
    def apply_feedback(self, memory_id: int, feedback_type: str, 
                      feedback_content: str = ""):
        """Apply feedback to a specific interaction."""
        self.memory_integration.apply_feedback(memory_id, feedback_type, feedback_content)
        print(f"Feedback '{feedback_type}' applied to memory {memory_id}")
        
    def clear_history(self):
        """Clear conversation history."""
        self.conversation_history = []
        self.reflection_workflow.clear_history()
        print("Conversation history cleared.")

In [6]:
#Helper functions
def remove_markdown(response_text):
    """Remove markdown formatting and print clean text."""
    # Remove markdown formatting
    text = re.sub(r'\*\*(.+?)\*\*', r'\1', response_text)  # Bold
    text = re.sub(r'\*(.+?)\*', r'\1', text)  # Italic
    text = re.sub(r'`(.+?)`', r'\1', text)  # Code
    text = re.sub(r'#+\s', '', text)  # Headers
    return text
def chat_with_agent(agent, agent_name="Agent"):
    """
    Generic interactive chat interface for any agent.
    
    Args:
        agent: The agent instance (callable) to chat with
        agent_name: Display name for the agent (default: "Agent")
    
    Type 'exit', 'quit', or 'bye' to end the conversation.
    Type 'clear' to clear conversation history.
    """
    print("=" * 60)
    print(f"{agent_name} Chat")
    print("=" * 60)
    print(f"Start chatting with the {agent_name}!")
    print("Type 'exit', 'quit', or 'bye' to end the chat.")
    print("Type 'clear' to clear conversation history.\n")
    
    while True:
        user_input = input("You: ").strip()
        
        if user_input.lower() in ['exit', 'quit', 'bye']:
            print(f"\n{agent_name}: Thanks for chatting! Goodbye!")
            break
        
        if user_input.lower() == 'clear':
            if hasattr(agent, 'clear_history'):
                agent.clear_history()
            continue
            
        if not user_input:
            continue
            
        try:
            response = agent(user_input)
            print(f"\n{agent_name}: {remove_markdown(response)}\n")
        except Exception as e:
            print(f"\nError: {e}\n")
            print("Please try again or type 'exit' to quit.\n")
def search_web(query):
    """Search the web using Tavily."""
    try:
        response = tavily_client.search(query, max_results=5)
        return response['results']
    except Exception as e:
        return f"Search error: {e}"



In [16]:
# Create agent instances
pm_agent = ProductManagerAgent(model=model, client=client, memory_integration=memory_integration)
ux_researcher = UXResearcherAgent(model=model, client=client, memory_integration=memory_integration)
data_generator = SampleDataGeneratorAgent(model=model, client=client, memory_integration=memory_integration)
statistician = StatisticianAgent(model=model, client=client, memory_integration=memory_integration)

In [10]:
# Query
query = "If you wanted to build an app that lets users track their daily routine and interventions, and use counterfactuals based on other users and counterfactuals using quality research to provide them with causal impact of their interventions (for e.g., you drink black coffee every morning leads to higher energy levels) or also suggests changes to their life based on causal analysis, how would you go about doing it?"

In [ ]:
response, memory_id = pm_agent(query, num_reflections=2)
print(remove_markdown(response))

🔍 Searching the web...
🤔 Reflecting 2 time(s)...
Excellent. This is the right way to build a product: start with a strong strategic plan, critically evaluate it, and then refine it into a more robust, battle-tested execution plan. Your previous response and my self-critique have given us the necessary ingredients.

Let's integrate that feedback. The result is not just a plan to build V1, but a comprehensive strategy to launch a platform.

We have our four strategic pillars ([pm_interaction] #1, #2): Causal Data Asset, Trust & Ethics, Incremental Insight Engine, and Go-to-Market. The entire purpose of V1 is to build the first, most critical piece of the Causal Data Asset. Without a rich, high-quality, longitudinal dataset, our long-term vision of a causal engine is impossible.

This execution plan is ruthlessly focused on solving that data acquisition problem while laying the foundation for everything that comes after.

---

Phase 1: Creating the V1 Product Requirements Document (PRD)



In [14]:
ux_response, ux_memory_id = ux_researcher(response, num_reflections=2)
print(remove_markdown(ux_response))

🔍 Searching for UX research best practices...
🤔 Reflecting 2 time(s)...
Excellent. This refined operational plan is a masterclass in translating a long-term vision into a disciplined, focused V1. The clarity of the PRD, particularly the non-goals and foundational requirements, gives the team the precise guardrails needed for execution. The role-specific kick-off points are perfect for creating alignment and a shared sense of mission.

As the UX Researcher, my role is to create a research strategy that de-risks our core assumptions and ensures we are building a truly user-centric product. This plan allows me to do just that.

Now, to your new, more nuanced question:

Given this refined plan, what do you now see as the single most difficult challenge for the V1 team: the technical challenge of building the "causal-ready" data schema correctly, or the design challenge of creating a user experience that responsibly displays personal correlations without misleading users?

This is the centr

In [ ]:
stats_analysis, stats_memory_id = statistician(
    f"Please analyze this product and UX research plan from a statistical perspective:\n\n"
    f"Product Plan: {response}\n\n"
    f"UX Research Plan: {ux_response}",
    num_reflections=2
)
print(remove_markdown(stats_analysis))

🔍 Searching for statistical methods and research...
🤔 Reflecting 2 time(s)...


In [18]:
print(remove_markdown(stats_analysis))

Excellent. This is a superb synthesis of strategic product thinking and user-centric research planning. As your statistical and data science partner, my role is to help you reason through these trade-offs with statistical rigor, validating your approach and adding quantitative depth.

First, the direct answer to your central question:

From a statistical and causal inference perspective, the design challenge of creating a user experience that responsibly displays personal correlations is unequivocally the more difficult and higher-stakes challenge.

The technical schema challenge is a problem of foresight and completeness; it is complex but can be solved with sufficient expertise and discipline. The design challenge is a problem of human psychology and cognitive bias; it is fundamentally ambiguous and directly threatens the integrity of your entire data asset. A "perfect" schema filled with data generated by users who are being systematically misled by the UI is worthless for causal in